### Continue, Break, and Else — Advanced Patterns (practical, not too much)

These examples expand your toolbox for loop control: guard-style `continue`, early-exit `break`, the underused `for/while ... else`, multi-level exits, and robust retry/cleanup flows.

## 1) Guard-style filtering with `continue` (skip bad records early)

In [1]:
rows = [
    {"id": 1, "name": "Alice",  "age": 30},
    {"id": 2, "name": "",       "age": 28},   # bad: empty name
    {"id": 3, "name": "Charlie", "age": None}, # bad: missing age
    {"id": 4, "name": "Dana",    "age": 40},
]

clean = []
for r in rows:
    # guard clauses → fast fail
    if not r.get("name"):
        continue
    if r.get("age") is None:
        continue
    clean.append({"id": r["id"], "name": r["name"].strip(), "age": int(r["age"])})

print(clean)  # [{'id': 1, ...}, {'id': 4, ...}]

[{'id': 1, 'name': 'Alice', 'age': 30}, {'id': 4, 'name': 'Dana', 'age': 40}]


## 2) Search with early `break` and `for ... else` fallback

In [2]:
names = ["Ada", "Grace", "Guido", "Ken", "Brian"]
target = "Guido"

for i, n in enumerate(names):
    if n == target:
        print(f"found {target} at index {i}")
        break
else:  # only runs if no break occurred
    print(f"{target} not found")

found Guido at index 2


## 3) Prime check using `for ... else` (classic, clean)

In [3]:
n = 97
if n < 2:
    print("not prime")
else:
    for d in range(2, int(n ** 0.5) + 1):
        if n % d == 0:
            print(f"composite: {n} = {d} * {n//d}")
            break
    else:
        print(f"prime: {n}")

prime: 97


## 4) Nested loops: breaking the inner vs both levels

In [4]:
grid = [
    [0, 0, 0],
    [0, 2, 0],
    [0, 0, 0],
]

# Break only the inner loop
for r in range(len(grid)):
    for c in range(len(grid[r])):
        if grid[r][c] == 2:
            print("found at:", (r, c))
            break
    # outer loop continues

# Break both levels with a flag (simple, explicit)
found = None
for r in range(len(grid)):
    for c in range(len(grid[r])):
        if grid[r][c] == 2:
            found = (r, c)
            break
    if found is not None:
        break
print("first 2 at:", found)

# Or: exit via a function to emulate a labeled break
def locate_two(G):
    for r in range(len(G)):
        for c in range(len(G[r])):
            if G[r][c] == 2:
                return (r, c)
    return None

print("locate_two:", locate_two(grid))

found at: (1, 1)
first 2 at: (1, 1)
locate_two: (1, 1)


## 5) `while ... else` for threshold scan (no early exit → else)

In [5]:
values = [1, 3, 5, 7]
i = 0
threshold = 10

while i < len(values):
    if values[i] >= threshold:
        print("hit threshold at index", i)
        break
    i += 1
else:  # never broke → no value ≥ threshold
    print("no value met threshold")

no value met threshold


## 6) Retry with `break/else`: success path vs all-attempts-failed path

In [6]:
import random

MAX_TRIES = 4
for attempt in range(1, MAX_TRIES + 1):
    result = random.random() < 0.4  # pretend flaky call
    if result:
        print(f"success on attempt {attempt}")
        break
else:
    print("all attempts failed; escalate or fallback")

success on attempt 1


## 7) Parsing with `continue`, sentinel `break`, and `for ... else` (no sentinel)

In [7]:
lines = [
    "# comment",
    " key = value  ",
    "",
    "END",
    "ignored after END",
]

pairs = {}
for raw in lines:
    line = raw.strip()
    if not line or line.startswith('#'):
        continue
    if line == "END":
        print("found sentinel END; stopping parse")
        break
    if "=" in line:
        k, v = (s.strip() for s in line.split("=", 1))
        pairs[k] = v
else:
    print("no END sentinel found; file may be truncated")

print(pairs)

found sentinel END; stopping parse
{'key': 'value'}


## 8) Cleanup always runs: `break` inside `try` with `finally`

In [8]:
resource_open = False
try:
    resource_open = True
    # loop that may break early
    for x in [1, 2, 3, 4]:
        if x == 3:
            print("condition met, breaking loop")
            break
        # work with resource
finally:
    if resource_open:
        resource_open = False
        print("resource closed in finally")

condition met, breaking loop
resource closed in finally


## 9) Multi-criteria filter: `continue` to keep the happy path left-aligned

In [9]:
emails = [
    " alice@example.com ",
    "bob@@example.com",
    "",
    "carol@example.org",
]

valid = []
for e in emails:
    e = e.strip().lower()
    if not e:
        continue
    if e.count("@") != 1:
        continue
    user, domain = e.split("@", 1)
    if not user or not domain or "." not in domain:
        continue
    valid.append(e)

print(valid)

['alice@example.com', 'carol@example.org']


## 10) Mini-exercises

1. Use `for ... else` to scan for the first negative in a list and print its index; if none, print "all non-negative".
2. Parse lines until a sentinel `STOP`; ignore blanks and `#` comments using `continue`.
3. In a nested loop, stop both levels as soon as the sum `i+j` equals a target (try both a flag and a function-return pattern).
4. Implement a fixed-attempt retry with `break/else` **and** a final `finally` cleanup section.